# Project 5 - Identify Fraud from Enron Email

## Background Information

Enron Corporation was an American energy, commodities, and services company. During 2000, Fortune named Enron "America's Most Innovative Company" and the end of 2001, it became bankrupt as a result of an accounting fraud. A lot of Enron data is now in the public domain, including tens of thousands of emails and financial data of executives.

For this project we will analyze the Enron email and financial data of 146 executives at Enron. We will identify a person of interest (POI), a person who was indicted for fraud. This report will cover the following:

1. Understanding the Enron Data.
2. Feature selection.
3. Algorithm analysis
4. Validation and Evaluation